In [1]:
#!pip install langchain neo4j openai wikipedia tiktoken langchain_openai
#!pip install -U python-dotenv

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

url="neo4j://192.168.68.84:7687/"
NEO_DB = os.getenv('NEO_DB')
username = os.getenv('NEO_USER')
password = os.getenv('NEO_PASS')


In [3]:
from langchain.graphs import Neo4jGraph
from langchain_openai import OpenAI
from langchain_openai import ChatOpenAI
from langchain.schema import AIMessage, HumanMessage, SystemMessage

# Import chat templates
from langchain.prompts import (
    ChatPromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

# Caching
from langchain.cache import InMemoryCache
from langchain.cache import SQLiteCache
from langchain.globals import set_llm_cache

from langchain.chains.openai_functions import (
    create_openai_fn_chain,
    create_structured_output_runnable,
    create_structured_output_chain
)

from langchain_community.graphs.graph_document import (
    Node as BaseNode,
    Relationship as BaseRelationship,
    GraphDocument,
)
from langchain.schema import Document
from typing import List, Dict, Any, Optional
from langchain.pydantic_v1 import Field, BaseModel
from tqdm import tqdm
from langchain.document_loaders import WikipediaLoader
from langchain.text_splitter import TokenTextSplitter


In [4]:
password

'password'

In [5]:
graph = Neo4jGraph(
    url=url,
    username=username,
    password=password
)

In [6]:


class Property(BaseModel):
  """A single property consisting of key and value"""
  key: str = Field(..., description="key")
  value: str = Field(..., description="value")

class Node(BaseNode):
    properties: Optional[List[Property]] = Field(
        None, description="List of node properties")

class Relationship(BaseRelationship):
    properties: Optional[List[Property]] = Field(
        None, description="List of relationship properties"
    )

class KnowledgeGraph(BaseModel):
    """Generate a knowledge graph with entities and relationships."""
    nodes: List[Node] = Field(
        ..., description="List of nodes in the knowledge graph")
    rels: List[Relationship] = Field(
        ..., description="List of relationships in the knowledge graph"
    )

In [7]:
def format_property_key(s: str) -> str:
    words = s.split()
    if not words:
        return s
    first_word = words[0].lower()
    capitalized_words = [word.capitalize() for word in words[1:]]
    return "".join([first_word] + capitalized_words)

def props_to_dict(props) -> dict:
    """Convert properties to a dictionary."""
    properties = {}
    if not props:
      return properties
    for p in props:
        properties[format_property_key(p.key)] = p.value
    return properties

def map_to_base_node(node: Node) -> BaseNode:
    """Map the KnowledgeGraph Node to the base Node."""
    properties = props_to_dict(node.properties) if node.properties else {}
    # Add name property for better Cypher statement generation
    properties["name"] = node.id.title()
    return BaseNode(
        id=node.id.title(), type=node.type.capitalize(), properties=properties
    )


def map_to_base_relationship(rel: Relationship) -> BaseRelationship:
    """Map the KnowledgeGraph Relationship to the base Relationship."""
    source = map_to_base_node(rel.source)
    target = map_to_base_node(rel.target)
    properties = props_to_dict(rel.properties) if rel.properties else {}
    return BaseRelationship(
        source=source, target=target, type=rel.type, properties=properties
    )

In [8]:
def get_system_template(allowed_nodes: Optional[List[str]] = None, allowed_rels: Optional[List[str]] = None):
    system_template = f"""
    # Knowledge Graph Instructions for GPT-4
    ## 1. Overview
    You are a top-tier algorithm designed for extracting information in structured formats to build a knowledge graph.
    - **Nodes** represent entities and concepts. They're akin to Wikipedia nodes.
    - The aim is to achieve simplicity and clarity in the knowledge graph, making it accessible for a vast audience.
    ## 2. Labeling Nodes
    - **Consistency**: Ensure you use basic or elementary types for node labels.
      - For example, when you identify an entity representing a person, always label it as **"person"**. Avoid using more specific terms like "mathematician" or "scientist".
    - **Node IDs**: Never utilize integers as node IDs. Node IDs should be names or human-readable identifiers found in the text.
    {'- **Allowed Node Labels:**' + ", ".join(allowed_nodes) if allowed_nodes else ""}
    {'- **Allowed Relationship Types**:' + ", ".join(allowed_rels) if allowed_rels else ""}
    ## 3. Handling Numerical Data and Dates
    - Numerical data, like age or other related information, should be incorporated as attributes or properties of the respective nodes.
    - **No Separate Nodes for Dates/Numbers**: Do not create separate nodes for dates or numerical values. Always attach them as attributes or properties of nodes.
    - **Property Format**: Properties must be in a key-value format.
    - **Quotation Marks**: Never use escaped single or double quotes within property values.
    - **Naming Convention**: Use camelCase for property keys, e.g., `birthDate`.
    ## 4. Coreference Resolution
    - **Maintain Entity Consistency**: When extracting entities, it's vital to ensure consistency.
    If an entity, such as "John Doe", is mentioned multiple times in the text but is referred to by different names or pronouns (e.g., "Joe", "he"), 
    always use the most complete identifier for that entity throughout the knowledge graph. In this example, use "John Doe" as the entity ID.  
    Remember, the knowledge graph should be coherent and easily understandable, so maintaining consistency in entity references is crucial. 
    ## 5. Strict Compliance
    Adhere to the rules strictly. Non-compliance will result in termination.
    """
    return system_template



def get_extraction_chain( llm, allowed_nodes: Optional[List[str]] = None, allowed_rels: Optional[List[str]] = None):
    human_template = """
    Use the given format to extract information from the following input: {input}
    Tip: Make sure to answer in the correct format
    """
    
    system_prompt = SystemMessagePromptTemplate.from_template(get_system_template(allowed_nodes, allowed_rels))
    human_prompt = HumanMessagePromptTemplate.from_template(human_template)
    
    chat_prompt = ChatPromptTemplate.from_messages([system_prompt, human_prompt])
    
    return create_structured_output_chain(output_schema=KnowledgeGraph, llm=llm, prompt=chat_prompt, verbose=False)
    

In [9]:
# def get_extraction_chain(
#     llm,
#     allowed_nodes: Optional[List[str]] = None,
#     allowed_rels: Optional[List[str]] = None
#     ):
#     prompt = ChatPromptTemplate.from_messages(
#         [(
#           "system",
#           f"""# Knowledge Graph Instructions for GPT-4
# ## 1. Overview
# You are a top-tier algorithm designed for extracting information in structured formats to build a knowledge graph.
# - **Nodes** represent entities and concepts. They're akin to Wikipedia nodes.
# - The aim is to achieve simplicity and clarity in the knowledge graph, making it accessible for a vast audience.
# ## 2. Labeling Nodes
# - **Consistency**: Ensure you use basic or elementary types for node labels.
#   - For example, when you identify an entity representing a person, always label it as **"person"**. Avoid using more specific terms like "mathematician" or "scientist".
# - **Node IDs**: Never utilize integers as node IDs. Node IDs should be names or human-readable identifiers found in the text.
# {'- **Allowed Node Labels:**' + ", ".join(allowed_nodes) if allowed_nodes else ""}
# {'- **Allowed Relationship Types**:' + ", ".join(allowed_rels) if allowed_rels else ""}
# ## 3. Handling Numerical Data and Dates
# - Numerical data, like age or other related information, should be incorporated as attributes or properties of the respective nodes.
# - **No Separate Nodes for Dates/Numbers**: Do not create separate nodes for dates or numerical values. Always attach them as attributes or properties of nodes.
# - **Property Format**: Properties must be in a key-value format.
# - **Quotation Marks**: Never use escaped single or double quotes within property values.
# - **Naming Convention**: Use camelCase for property keys, e.g., `birthDate`.
# ## 4. Coreference Resolution
# - **Maintain Entity Consistency**: When extracting entities, it's vital to ensure consistency.
# If an entity, such as "John Doe", is mentioned multiple times in the text but is referred to by different names or pronouns (e.g., "Joe", "he"),
# always use the most complete identifier for that entity throughout the knowledge graph. In this example, use "John Doe" as the entity ID.
# Remember, the knowledge graph should be coherent and easily understandable, so maintaining consistency in entity references is crucial.
# ## 5. Strict Compliance
# Adhere to the rules strictly. Non-compliance will result in termination.
#           """),
#             ("human", "Use the given format to extract information from the following input: {input}"),
#             ("human", "Tip: Make sure to answer in the correct format"),
#         ])
#     return create_structured_output_chain(KnowledgeGraph, llm, prompt, verbose=False)

In [10]:
def extract_and_store_graph(
    document: Document,
    nodes:Optional[List[str]] = None,
    rels:Optional[List[str]]=None) -> None:

    llm = ChatOpenAI(model="gpt-3.5-turbo-16k", temperature=0)
    
    # Extract graph data using OpenAI functions
    extract_chain = get_extraction_chain(llm, nodes, rels)
    data = extract_chain.invoke(document.page_content)['function']
    
    # Construct a graph document
    graph_document = GraphDocument(
      nodes = [map_to_base_node(node) for node in data.nodes],
      relationships = [map_to_base_relationship(rel) for rel in data.rels],
      source = document
    )
    
    # Store information into a graph
    graph.add_graph_documents([graph_document])

In [11]:

# Read the wikipedia article
raw_documents = WikipediaLoader(query="Walt Disney").load()
# Define chunking strategy
text_splitter = TokenTextSplitter(chunk_size=2048, chunk_overlap=24)

# Only take the first the raw_documents
documents = text_splitter.split_documents(raw_documents[:3])

In [12]:
# Specify which node labels should be extracted by the LLM
allowed_nodes = ["Person", "Company", "Location", "Event", "Movie", "Service", "Award"]

for i, d in tqdm(enumerate(documents), total=len(documents)):
    extract_and_store_graph(d, allowed_nodes)
    

  0%|          | 0/3 [00:00<?, ?it/s]/opt/conda/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `create_structured_output_chain` was deprecated in LangChain 0.1.1 and will be removed in 0.2.0. Use create_structured_output_runnable instead.
  warn_deprecated(
100%|██████████| 3/3 [02:27<00:00, 49.15s/it]


In [ ]:
# Query the knowledge graph in a RAG application
from langchain.chains import GraphCypherQAChain

In [ ]:
graph.refresh_schema()

cypher_chain = GraphCypherQAChain.from_llm(
    graph=graph,
    cypher_llm=ChatOpenAI(temperature=0, model="gpt-4"),
    qa_llm=ChatOpenAI(temperature=0, model="gpt-3.5-turbo"),
    validate_cypher=True, # Validate relationship directions
    verbose=True
)

In [ ]:
cypher_chain.invoke({"query": "When was Walter Elias Disney born?"})